# Neighborhood components Analysis

## Introduction

This notebook has been made to illustrate the [paper](https://cs.nyu.edu/~roweis/papers/ncanips.pdf) Jacob Goldberger, Sam Roweis, Geoff Hinton, Ruslan Salakhutdinov. I strongly recommend reading this paper before playing that notebook.

This paper presents a novel non parametric method for learning the Mahalanobis distance. This method addresses the two major issues encountered with KNN classification :
- the computational isssue by reducing the dimension of the problem
- the definition of a "near" neighbor by providing a new quadratic distance metric

To illustrate this paper and show its performances, we will use the NIST dataset for the examples.

<div class="alert alert-block alert-danger">
Some of the exercices in this notebook can be hard if you haven't read the relative paper. I recommend not spending too much time stuck on a question, load the solution if you spend more than 5 minutes stuck and most importantly try and understand the idea of what we are doing. 
    
## Don't stay stuck, don't get lost !
</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
digits = datasets.load_digits()
y = digits.data
d = digits.target
X_train, X_test, c_train, c_test = train_test_split(y, d, test_size=0.3)

D = X_train.shape[1]
n = X_train.shape[0]

print(f'Number of fearures (D) : {D}')
print(f'Number of vectors (n) : {n}')

## Stochastic Nearest Neighbours for Distance Metric Learning

We begin with a labeled data set consisting of n real-valued input vectors $x_1$, . . . , $x_n$ in $R^D$
and corresponding class labels $c_1$, ..., $c_n$

We here aim at optimizing the Leave-One-Out (LOO) classification error on the training data. First, we restrict ourselves to learning  Mahalanobis (quadratic) distance metrics, which can always be represented by symmetric positive semi-definite matrices. Then, if we denote the transformation by a matrix $A$ we are effectively learning a metric $Q = A^TA$ such that $d(x, y) = (x − y)^T Q(x − y) = (Ax − Ay)^T (Ax − Ay)$

<div class="alert alert-warning">
    
**Exercice:** Initialize a matrix $A$
</div>

<div class="alert alert-block alert-info">
<b>Tip:</b> To get better performances, you can initialize A to the identity matrix.</div>

In [ ]:
A = ...

In [ ]:
# %load solutions/matrix_A.py

To make the LOO classification error continuous, we have to make the neighbor selection continuous. To do so, each point $i$ selects another point $j$ as its neighbour with some probability $p_{ij}$, and inherits its class label from the point it selects. After applying a softmax, we have :

\begin{equation*}
p_{ij} = \frac{\exp(−{\lVert Ax_i − Ax_{j}\rVert}^2)}{\sum_{k\ne i}{\exp(−{\lVert Ax_i − Ax_{k}\rVert}^2))}}, \;\; p_{ii} = 0
\end{equation*}

<div class="alert alert-warning">
    
**Exercice:** Compute $p_{ij}$
</div>

In [ ]:
p_ij = ...

In [ ]:
# %load solutions/p_ij.py

We can then compute the probability $p_i$ that point $i$ is going to be correctly classified :

\begin{equation*}
p_{i} = \sum_{i \in C_i}{p_{ij}}, \;\;\; C_{i} = \{j|c_i = c_j\}
\end{equation*}

<div class="alert alert-warning">
    
**Exercice:** Compute $p_{i}$
</div>

In [ ]:
same_class_mask = ...
masked_p_ij = ...
p = ...

In [ ]:
# %load solutions/p_i.py

As we aim at correctly classifying a maximum of points, our objective function is :

\begin{equation*}
f(A) = \sum_{i}{\sum_{j \in C_i}{p_{ij}}} = \sum_{i}{p_i} 
\end{equation*}

<div class="alert alert-warning">
    
**Exercice:** Compute the objective function
</div>

In [ ]:
loss = ...

In [ ]:
# %load solutions/objective_function.py

Differentiating $f$ with respect to the transformation matrix $A$ yields a gradient rule which
we can use for learning :

\begin{equation*}
\frac{∂f}{∂A} = 2A \sum_{i}{\left(p_i \sum_{k}{p_{ik}x_{ik}x_{ik}^T} - \sum_{j \in C_i}{p_{ij}x_{ij}x_{ij}^T}\right)}
\end{equation*}


<div class="alert alert-warning">
    
**Exercice:** Implement this gradient rule
</div>

In [ ]:
# %load solutions/gradient.py

<div class="alert alert-warning">
    
**Exercice:** Let's put all that in a function to compute the gradient
</div>

In [ ]:
def grad(A,x, same_class_mask):
    ...

In [ ]:
# %load solutions/gradient_function.py

<div class="alert alert-warning">
    
**Exercice:** Minimise the loss
</div>

In [ ]:
# %load solutions/optimizer.py

Now that we've learned the transformation, we can evaluate its performance 

<div class="alert alert-warning">
    
**Exercice:** Compare the performance of a KNN on the raw data with one of a KNN on the transformed data
</div>

In [ ]:
# %load solutions/performance.py

## Low Rank Distance Metrics and Nonsquare Projection

The other main advantage to NCA is its performance in reducing the dimensionnality of the input data. It can be very useful to speed up computations in case of large input data dimensionnality.

To do so, all we need is to reduce the output size of a from the number of features to the dimentionnality we want.

<div class="alert alert-warning">
    
**Exercice:** Take all the work you have done so far and modify the output size of A to 2 to reduce the dimensionnality of the input vectors
</div>

<div class="alert alert-block alert-info">
<b>Tip:</b> To get better performances, you can initialize A to PCA transformation matrix.</div>

In [ ]:
# %load solutions/rectangle_matrix.py

<div class="alert alert-warning">
    
**Exercice:** Check the performance of a KNN on the data after a PCA transformation with one of a KNN on the data after a NCA transformation
</div>

In [ ]:
# %load solutions/rectangle_performance.py

You can now see how much more efficient the NCA transformation is compared to a classic PCA. Here is a chart from the original paper showing the performance of NCA :

<figure>
  <img src="images/performance.png" alt="Trulli" style="width:100%">
  <figcaption>KNN classification accuracy (left train, right test) on UCI datasets balance, ionosphere, iris, wine and housing and on the USPS handwritten digits. Results are averages
over 40 realizations of splitting each dataset into training (70%) and testing (30%) subsets
(for USPS 200 images for each of the 10 digit classes were used for training and 500 for
testing). Top panels show distance metric learning (square A) and bottom panels show
linear dimensionality reduction down to d = 2.</figcaption>
</figure>

Here is an illustration of the transformation made by NCA compared to PCA and LDA :

<figure>
  <img src="images/transformation_illustration.png" alt="Trulli" style="width:100%">
  <figcaption>Dataset visualization results of PCA, LDA and NCA applied to (from top) the
“concentric rings”, “wine”, “faces” and “digits” datasets. The data are reduced from their
original dimensionalities (D=3,D=13,D=560,D=256 respectively) to the d=2 dimensions
show.</figcaption>
</figure>

## Additional information

### scikit-learn implementation

Since this paper has been published, it has been implemented in scikit-learn. [Here](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NeighborhoodComponentsAnalysis.html#sklearn.neighbors.NeighborhoodComponentsAnalysis.fit) is the documentation of the class

<div class="alert alert-warning">
    
**Exercice:** Use the scikit-learn implementation on the digits dataset.
</div>

In [ ]:
# %load solutions/sklearn_implementation.py

### Let's play with a little illustration

Let's now try and reproduce part of the illustration above with a part of the digits dataset

In [ ]:
digits = datasets.load_digits(n_class=5) #We reduce the number of selected digits to reduce the number of classes
y = digits.data
d = digits.target
X_train, X_test, c_train, c_test = train_test_split(y, d, test_size=0.3)

D = X_train.shape[1]
n = X_train.shape[0]

print(f'Number of fearures (D) : {D}')
print(f'Number of vectors (n) : {n}')

<div class="alert alert-warning">
    
**Exercice:** Plot the data projected un a 2D space with PCA, LDA and NCA
</div>

In [ ]:
...

X_test_pca = ...
X_test_lda = ...
X_test_nca = ...

fig, ax = plt.subplots(1, 3, figsize=(20, 7))
ax[0].scatter(X_test_pca[:,0], X_test_pca[:,1], c=c_test)
ax[0].set_xlabel('PCA')
ax[1].scatter(X_test_lda[:,0], X_test_lda[:,1], c=c_test)
ax[1].set_xlabel('LDA')
ax[2].scatter(X_test_nca[:,0], X_test_nca[:,1], c=c_test)
ax[2].set_xlabel('NCA')
plt.show()

In [ ]:
# %load solutions/illustration.py

You can play with the number of digits you keep and see how different and better the NCA transformation is compared to the others.

### Easier to compute objective function

Maximizing the objective function $f(A)$ is equivalent to minimizing the $L_1$ norm between
the true class distribution (having probability one on the true class) and the stochastic class
distribution induced by $p_{ij}$ via $A$. A natural alternative distance is the KL-divergence which
induces the following objective function:

\begin{equation*}
g(A) = \sum_{i}{log\left(\sum_{j \in C_i}{p_{ij}}\right)} = \sum_{i}{log(p_i)} 
\end{equation*}

Maximizing this objective would correspond to maximizing the probability of obtaining a
perfect (error free) classification of the entire training set. The gradient of $g(A)$ is even
simpler than that of $f(A)$:

\begin{equation*}
\frac{∂g}{∂A} = 2A \sum_{i}{\left(\sum_{k}{p_{ik}x_{ik}x_{ik}^T} - \frac{\sum_{j \in C_i}{p_{ij}x_{ij}x_{ij}^T}}{\sum_{j \in C_i}{p_{ij}}}\right)}
\end{equation*}

### Other dimension reduction algorithms (relation to previous other works)

#### LDA

Fisher introduced in his [paper](https://www.comp.tmu.ac.jp/morbier/R/Fisher-1936-Ann._Eugen.pdf) in 1936, the Fisher's linear discriminant which is the base for Linear discriminant analysis (LDA). It is a method used to find a linear combination of features that characterizes or separates two or more classes. The resulting combination may be used for classification or dimensionality reduction.

LDA dimensionality reduction matrix can be used as an initialization of the matrix A in NCA in order to find a better local optimum.

#### RCA

Relevant components analysis, introduced in 2002 by Noam Shental, Tomer Hertz, Daphna Weinshall, and Misha Pavel in this [paper](https://link.springer.com/content/pdf/10.1007/3-540-47979-1_52.pdf) and later used in this [paper](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.5.3038&rep=rep1&type=pdf) is a method which aims at finding a transformation that amplifies relevant variability and suppresses irrelevant variability. 

Neither of these method are as efficient as NCA in dimension reduction for classifying data.